In [1]:
// Import the API, Keyring and some utility functions
const { ApiPromise } = require('@polkadot/api');
const { Keyring } = require('@polkadot/keyring');

const BOB = '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty';

// Instantiate the API
const api = await ApiPromise.create();

// Constuct the keying after the API (crypto has an async init)
const keyring = new Keyring({ type: 'sr25519' });

// Add Alice to our keyring with a hard-deived path (empty phrase, so uses dev)
const alice = keyring.addFromUri('//Alice');

In [2]:
const ADDR = '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty';
// Subscribe to balance changes for our account
const unsub = await api.query.system.account(ADDR, ({ nonce, data: balance }) => {
  console.log(`free balance is ${balance.free} with ${balance.reserved} reserved and a nonce of ${nonce}`);
});

free balance is 999998900000000012345 with 1100000000000000 reserved and a nonce of 0


In [3]:
unsub() // 停止订阅并清除所有基础RPC连接

In [6]:
// 通过单个RPC调用来跟踪许多状态条目，而不是为每个单个项目进行调用
// 一次检索多个帐户的余额
const ADDR1='5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty';
const ADDR2='5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY'; // alice
// Subscribe to balance changes for 2 accounts, ADDR1 & ADDR2 (already defined)
const unsub = await api.query.system.account.multi([ADDR1, ADDR2], (balances) => {
  const [{ data: balance1 }, { data: balance2 }] = balances;

  console.log(`The balances are ${balance1.free} and ${balance2.free}`);
});

The balances are 999998900000000012345 and 999998895739040142956


In [7]:
unsub() 

In [8]:
// Retrieve a snapshot of the validators
const validatorKeys = await api.query.session.validators.keys();
validatorKeys

TypeError: exports.api.query.session.validators.keys is not a function
    at evalmachine.<anonymous>:4:66
    at evalmachine.<anonymous>:8:3
    at sigintHandlersWrap (vm.js:288:15)
    at Script.runInThisContext (vm.js:120:14)
    at Object.runInThisContext (vm.js:329:38)
    at Object.execute (/Users/xiaofeiwu/.nvm/versions/node/v10.17.0/lib/node_modules/tslab/dist/executor.js:162:38)
    at JupyterHandlerImpl.handleExecuteImpl (/Users/xiaofeiwu/.nvm/versions/node/v10.17.0/lib/node_modules/tslab/dist/jupyter.js:219:38)
    at count.execQueue.add (/Users/xiaofeiwu/.nvm/versions/node/v10.17.0/lib/node_modules/tslab/dist/jupyter.js:177:57)


### 多查询，不同类型
前面的.multi示例假定我们对相同类型进行查询，即检索了多个帐户的余额。但是，还需要检索各种不同的类型，例如，除了特定帐户的现时和余额外，我们还希望跟踪区块时间戳。为此，该api具有特定的api.queryMulti接口，可用于执行此查询

In [9]:
// Subscribe to the timestamp, our index and balance
const unsub = await api.queryMulti([
  api.query.timestamp.now,
  [api.query.system.account, ADDR]
], ([now, { nonce, data: balance }]) => {
  console.log(`${now}: balance of ${balance.free} and a nonce of ${nonce}`);
});

1598648640003: balance of 999998900000000012345 and a nonce of 0
1598648643004: balance of 999998900000000012345 and a nonce of 0
1598648646002: balance of 999998900000000012345 and a nonce of 0
1598648649000: balance of 999998900000000012345 and a nonce of 0


In [10]:
unsub()

In [11]:
// 检索当前余额以及特定帐户上一个区块的余额
// Retrieve the current block header
const lastHdr = await api.rpc.chain.getHeader();

// Retrieve the balance at both the current and the parent hashes
const [{ data: balanceNow }, { data: balancePrev }] = await Promise.all([
  api.query.system.account.at(lastHdr.hash, ADDR),
  api.query.system.account.at(lastHdr.parentHash, ADDR)
]);

// Display the difference
console.log(`The delta was ${balanceNow.free.sub(balancePrev.free)}`);

The delta was 0


In [12]:
// .at查询都是单发的，即没有订阅选项，因为前一个块的状态应该是静态的
// Retrieve the timestamp for the previous block
const momentPrev = await api.query.timestamp.now.at(lastHdr.parentHash);
momentPrev

<BN: 17436e3e4b8>


In [5]:
// 查询从特定历史块开始直到指定块或最新块的状态。
// 这是通过.range([from, to?], <...opt params>): [Hash, Type][]查询完成的
// Retrieve the current block header
const lastHdr = await api.rpc.chain.getHeader();
const startHdr = await api.rpc.chain.getBlockHash(lastHdr.number.unwrap().subn(500));

// retrieve the range of changes
// ? Error: Account is a Map and requires one argument
// const changes = await api.query.system.account.range([startHdr]);

// changes.forEach(([hash, value]) => {
//   console.log(hash.toHex(), value.toHuman());
// });

2020-08-29 05:15:56          API-WS: disconnected from ws://127.0.0.1:9944 code: '1006' reason: 'Connection dropped by remote peer.'
2020-08-29 05:15:57          API-WS: disconnected from ws://127.0.0.1:9944 code: '1006' reason: 'connection failed'
2020-08-29 05:15:58          API-WS: disconnected from ws://127.0.0.1:9944 code: '1006' reason: 'connection failed'
2020-08-29 05:15:59          API-WS: disconnected from ws://127.0.0.1:9944 code: '1006' reason: 'connection failed'
2020-08-29 05:16:00          API-WS: disconnected from ws://127.0.0.1:9944 code: '1006' reason: 'connection failed'
2020-08-29 05:16:01          API-WS: disconnected from ws://127.0.0.1:9944 code: '1006' reason: 'connection failed'
2020-08-29 05:16:02          API-WS: disconnected from ws://127.0.0.1:9944 code: '1006' reason: 'connection failed'
2020-08-29 05:16:03          API-WS: disconnected from ws://127.0.0.1:9944 code: '1006' reason: 'connection failed'
2020-08-29 05:16:04          API-WS: disconnected from 